In [1]:
# 載入需要的套件
import os
import pandas as pd
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 設定 data_path
dir_data = './data/'

# 讀取資料檔
f_data = os.path.join(dir_data, 'train.csv')
data = pd.read_csv(f_data,header = None)
data = data.append(data)
print(data.shape)

# 讀取資料檔
f_label = os.path.join(dir_data, 'trainLabels.csv')
label = pd.read_csv(f_label,header = None)
label = label.append(label)
print(label.shape)

# from sklearn import preprocessing
# data = preprocessing.scale(data)

from sklearn.preprocessing import StandardScaler, Normalizer

norm = Normalizer()
data = norm.fit_transform(data)

(2000, 40)
(2000, 1)


In [10]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.1, random_state=4)
# 建立一個線性回歸模型
LogisticRegression = linear_model.LogisticRegression()

# 將訓練資料丟進去模型訓練
LogisticRegression.fit(x_train, y_train)

# 將測試資料丟進模型得到預測結果
y_pred = LogisticRegression.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Logistic: ", acc)

[[-0.10185882 -0.06715206 -0.0936666  ...  0.01845507 -0.38332155
  -0.28676844]
 [ 0.00197568 -0.02785185  0.11317489 ...  0.01128098 -0.05162831
   0.13085704]
 [ 0.1353222  -0.05013337 -0.08342005 ...  0.04612763  0.10913548
   0.18041458]
 ...
 [ 0.01992434  0.07963108  0.05467088 ... -0.03370306  0.09231461
   0.16359838]
 [ 0.06602803 -0.0762501  -0.10961667 ...  0.01356526  0.03124672
   0.13924554]
 [-0.03045145  0.04224076 -0.03888053 ...  0.21521913 -0.12597119
   0.11795832]]
Logistic:  0.835


In [11]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(x_train,y_train)
y_pred = knn_model.predict(x_test)
print(y_pred)
print('KNN',accuracy_score(y_test, y_pred))
print('KNN',cross_val_score(knn_model,y_test, y_pred, cv=10).mean())

#SVM
from sklearn.svm import SVC

svc_model = SVC(gamma = 'auto')
svc_model.fit(x_train,y_train)
y_pred = svc_model.predict(x_test)
print('SVM',accuracy_score(y_test, y_pred))
print('SVM',cross_val_score(svc_model,y_test, y_pred, cv=10).mean())


#DECISON TREE
from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier()
dtree_model.fit(x_train,y_train)
y_pred = dtree_model.predict(x_test)
print('Decision Tree',accuracy_score(y_test, y_pred))
print('Decision',cross_val_score(dtree_model,y_test, y_pred, cv=10).mean())

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(n_estimators = 100,random_state = 99)
rfc_model.fit(x_train,y_train)
y_pred = rfc_model.predict(x_test)
print('Random Forest',accuracy_score(y_test, y_pred))
print('Random Forest',cross_val_score(rfc_model,y_test, y_pred, cv=10).mean())

[0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1
 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 0
 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1
 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0
 0 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0
 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1]
KNN 0.925
KNN 0.9238972431077694
SVM 0.835
SVM 0.8340100250626566
Decision Tree 1.0
Decision 1.0
Random Forest 0.99
Random Forest 0.9899749373433584


In [4]:
test_data = os.path.join(dir_data, 'test.csv')
f_test_data = pd.read_csv(test_data,header = None)
print(f_test_data.shape)
predict = rfc_model.predict(f_test_data)
print(predict)

(9000, 40)
[1 1 1 ... 0 0 1]


In [5]:
df = pd.DataFrame({"Solution":predict})
df.index += 1
df.to_csv("day48_predict.csv",index = True, sep = ",",index_label = "Id")

In [6]:
knn = KNeighborsClassifier()
n_neighbors=[3,5,6,7,8,9,10]
param_grid = dict(n_neighbors=n_neighbors)

grid_search_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv = 10, n_jobs=-1,scoring='accuracy').fit(x_train,y_train.values.ravel())
knn_best = grid_search_knn.best_estimator_
print('KNN final best Score', grid_search_knn.best_score_)
print('KNN final best Params',grid_search_knn.best_params_)
print('KNN Accuracy',cross_val_score(knn_best,y_test, y_pred, cv=10).mean())

KNN final best Score 0.92
KNN final best Params {'n_neighbors': 5}
KNN Accuracy 0.9899749373433584


In [7]:
test_data = os.path.join(dir_data, 'test.csv')
f_test_data = pd.read_csv(test_data,header = None)
print(f_test_data.shape)
predict = grid_search_knn.predict(f_test_data)
print(predict)

(9000, 40)
[1 0 1 ... 1 0 1]


In [8]:
df = pd.DataFrame({"Solution":predict})
df.index += 1
df.to_csv("day48_predict.csv",index = True, sep = ",",index_label = "Id")